### conda作成

In [ ]:
# ! conda create -n [name] python=3.8
# ! conda install jupyter

##### req

In [ ]:
# xenhan
! pip install zenhan

In [ ]:
# pyknp
! pip install pyknp

In [ ]:
#　sentencepiece
! pip install sentencepiece

In [ ]:
# (optional) tensorboardX
! pip install tensorboardX

In [ ]:
# fairseq
! git clone https://github.com/utanaka2000/fairseq
! pip install --editable fairseq

https://github.com/utanaka2000/fairseq/blob/japanese_bart_pretrained_model/jaBART_preprocess.py 

↑ここからjaBART_preprocess.py持ってくる。

In [21]:
# jumanpp-2.0.0-rc3
! wget https://github.com/ku-nlp/jumanpp/releases/download/v2.0.0-rc3/jumanpp-2.0.0-rc3.tar.xz
! tar xvf jumanpp-2.0.0-rc3.tar.xz

# jumanpp-2.0.0-rc3/libs/catch.hpp                  ~6609~  -> char FatalConditionHandler::altStackMem[32768] = {};
# jumanpp-2.0.0-rc3/src/util/serialization_test.cc  ~top~   -> #include <limits>

! mkdir jumanpp-2.0.0-rc3/bld
! cd jumanpp-2.0.0-rc3/bld && cmake .. -DCMAKE_BUILD_TYPE=Release -DCMAKE_INSTALL_PREFIX='where'
! cd jumanpp-2.0.0-rc3/bld && make install -j 4

In [6]:
# BART large v2.0 (3.7G)
! wget http://lotus.kuee.kyoto-u.ac.jp/nl-resource/JapaneseBARTPretrainedModel/japanese_bart_large_2.0.tar.gz
! tar -zxvf japanese_bart_large_2.0.tar.gz

In [ ]:
# preprocess
# jumanpp = Juman(command = 'jumanpp-2.0.0-rc3/bld/where/bin/jumanpp')

TRAIN_SRC = 'main_data/train_src.txt'
TRAIN_TGT = 'main_data/train_tgt.txt'
VALID_SRC = 'main_data/val_src.txt'
VALID_TGT = 'main_data/val_tgt.txt'
TEST_SRC = 'main_data/test_src.txt'
TEST_TGT = 'main_data/test_tgt.txt'

# 以下、モデルに応じて書き換え
SENTENCEPIECE_MODEL = 'japanese_bart_large_2.0/sp.model'
DICT = 'japanese_bart_large_2.0/dict.txt'
DATASET_DIR = 'large2_dataset_dir'
! mkdir $DATASET_DIR

! cat $TRAIN_SRC | python3 jaBART_preprocess.py --bpe_model $SENTENCEPIECE_MODEL --bpe_dict $DICT > $DATASET_DIR/train.src-tgt.src
! cat $TRAIN_TGT | python3 jaBART_preprocess.py  --bpe_model $SENTENCEPIECE_MODEL --bpe_dict $DICT > $DATASET_DIR/train.src-tgt.tgt
! cat $VALID_SRC | python3 jaBART_preprocess.py --bpe_model $SENTENCEPIECE_MODEL --bpe_dict $DICT > $DATASET_DIR/valid.src-tgt.src
! cat $VALID_TGT | python3 jaBART_preprocess.py --bpe_model $SENTENCEPIECE_MODEL --bpe_dict $DICT > $DATASET_DIR/valid.src-tgt.tgt
! cat $TEST_SRC | python3 jaBART_preprocess.py --bpe_model $SENTENCEPIECE_MODEL --bpe_dict $DICT > $DATASET_DIR/test.src-tgt.src
! cat $TEST_TGT | python3 jaBART_preprocess.py --bpe_model $SENTENCEPIECE_MODEL --bpe_dict $DICT > $DATASET_DIR/test.src-tgt.tgt
! cp $DICT $DATASET_DIR/dict.src.txt
! cp $DICT $DATASET_DIR/dict.tgt.txt
print('done!')

In [17]:
# fairseq/fairseq/data/indexed_dataset.py 
# 98, 301 np.float -> float

In [ ]:
# finetune

# 以下、モデルに応じて書き換え
# DATASET_DIR = 'large2_dataset_dir'
# BART = 'bart_large'
# PRETRAINED_MODEL = 'japanese_bart_large_2.0/bart_model.pt'
# SAVE_MODEL_DIR = 'large2_model_save'
# ! mkdir large2_model_save
# TENSORBOARD_DIR = 'large2_tensorboard'
# ! mkdir large2_tensorboard

# --restore-fileのcontentが必要みたい。
! nohup fairseq-train large2_dataset_dir --arch bart_large --restore-file content/japanese_bart_large_2.0/bart_model.pt \
    --save-dir large2_model_save --tensorboard-logdir large2_tensorboard \
    --task translation_from_pretrained_bart --source-lang src --target-lang tgt \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.2 --dataset-impl raw \
    --optimizer adam --adam-eps 1e-06 --adam-betas '{0.9, 0.98}' --lr-scheduler polynomial_decay --lr 5e-05 --min-lr -1 \
    --warmup-updates 2500 --total-num-update 10000 --weight-decay 0.3 \
    --max-tokens 1024 --update-freq 2 --save-interval -1 --no-epoch-checkpoints --log-format simple --log-interval 2 \
    --reset-optimizer --reset-meters --reset-dataloader --reset-lr-scheduler  --save-interval-updates 5000 \
    --ddp-backend no_c10d --max-update 20000 \
    --encoder-normalize-before --decoder-normalize-before --langs ja --prepend-bos --patience 5

In [ ]:
# text-generate

# DATASET_DIR = 'large2_dataset_dir'
# RESULT = 'large2_ja2ja'
# FINETUNED_MODEL = 'large2_model_save/checkpoint_best.pt'

! fairseq-generate dataset_dir --path large2_model_save/checkpoint_best.pt --task translation_from_pretrained_bart \
    --dataset-impl raw --gen-subset test -s src -t tgt --max-sentences 64 --langs ja --prepend-bos > large2_ja2ja

! grep ^H large2_ja2ja | LC_ALL=C sort -V | cut -f 3- | sed 's/<<unk>>/<unk>/g' | sed 's/▁//g' > large2_ja2ja.pred
! grep ^S large2_ja2ja | LC_ALL=C sort -V | cut -f 2- | sed 's/<<unk>>/<unk>/g' | sed 's/▁//g' > large2_ja2ja.src
! grep ^T large2_ja2ja | LC_ALL=C sort -V | cut -f 2- | sed 's/<<unk>>/<unk>/g' | sed 's/▁//g' > large2_ja2ja.tgt

In [ ]:
# text-generate

# DATASET_DIR = 'large2_dataset_dir'
# RESULT = 'large2_ja2ja'
# FINETUNED_MODEL = 'large2_model_save/checkpoint_best.pt'

! fairseq-generate large2_dataset_dir --path large2_model_save/checkpoint_best.pt --task translation_from_pretrained_bart \
    --dataset-impl raw --gen-subset test -s src -t tgt --max-sentences 64 --langs ja --prepend-bos > large2_ja2ja

! grep ^H large2_ja2ja | LC_ALL=C sort -V | cut -f 3- | sed 's/<<unk>>/<unk>/g' | sed 's/▁//g' > large2_ja2ja.pred
! grep ^S large2_ja2ja | LC_ALL=C sort -V | cut -f 2- | sed 's/<<unk>>/<unk>/g' | sed 's/▁//g' > large2_ja2ja.src
! grep ^T large2_ja2ja | LC_ALL=C sort -V | cut -f 2- | sed 's/<<unk>>/<unk>/g' | sed 's/▁//g' > large2_ja2ja.tgt